In [ ]:
from IPython.display import clear_output

## Set up a virtual display

Mujoco rendering requires a display(even if we want to just generate rgb_array frames). A colab notebook does not have a display. Therefore we need to account for that by creating a virtual display

In [ ]:
%%capture
!apt install -y python3-opengl
!apt install -y ffmpeg
!apt install -y xvfb
!pip3 install pyvirtualdisplay

clear_output()

In [ ]:
from pyvirtualdisplay import Display

display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
%pip install gymnasium[mujoco] stable-baselines3

clear_output()

In [ ]:
# %pip install numpy matplotlib

# Content

In this demo we will use the Soft Actor Critic algorithm to solve the **"Humanoid Standup"** Environment. We will use Stable baselines3's implementation of SAC.

In the humanoid standup environment, a humanoid figure who is lying on the ground needs to get up and the actions are how much force to apply on humanoid's rotors and hunges. More info can be found [here](https://gymnasium.farama.org/environments/mujoco/humanoid_standup/)

![Pusher Image](https://gymnasium.farama.org/_images/humanoid_standup.gif)

In [ ]:
import numpy as np

import gymnasium as gym
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.noise import NormalActionNoise

from IPython.display import clear_output

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [ ]:
def frames_to_video(frames, fps=24):
    fig = plt.figure(figsize=(frames[0].shape[1] / 100, frames[0].shape[0] / 100), dpi=100)
    ax = plt.axes()
    ax.set_axis_off()

    if len(frames[0].shape) == 2:  # Grayscale image
        im = ax.imshow(frames[0], cmap='gray')
    else:  # Color image
        im = ax.imshow(frames[0])

    def init():
        if len(frames[0].shape) == 2:
            im.set_data(frames[0], cmap='gray')
        else:
            im.set_data(frames[0])
        return im,

    def update(frame):
        if len(frames[frame].shape) == 2:
            im.set_data(frames[frame], cmap='gray')
        else:
            im.set_data(frames[frame])
        return im,

    interval = 1000 / fps
    anim = FuncAnimation(fig, update, frames=len(frames), init_func=init, blit=True, interval=interval)
    plt.close()
    return HTML(anim.to_html5_video())

## Creating the environment

SB3 vectorized environments for multiple environments and faster training

In [ ]:
# Create the environment
make_env = lambda: gym.make('HumanoidStandup-v4')
env = DummyVecEnv([make_env for _ in range(4)])  # adjust accoring to available ram
num_actions = env.action_space.shape[0]

In [ ]:
noise_mean = np.array([0.0] * num_actions)
noise_std = np.array([0.1] * num_actions)

## Training the model

In [ ]:
model = SAC("MlpPolicy",
            env,
            verbose=1,
            action_noise=NormalActionNoise(noise_mean, noise_std),  # noise for exploration
            learning_rate=1e-3)

In [ ]:
clear_output()
model.learn(total_timesteps=int(1e6), progress_bar=True)

## Evaluating and visualizing the performance of trained model

In [ ]:
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward:.2f}")

In [ ]:
t_env = DummyVecEnv([lambda: gym.make('HumanoidStandup-v4', render_mode="rgb_array")])
state = t_env.reset()
frames = []

while True:
    action, _ = model.predict(state)
    state_next, r, done, info = t_env.step(action)
    frames.append(t_env.render())
    state = state_next
    if done.all():
        break

t_env.close()

In [ ]:
frames_to_video(frames)